This is a primer for the pandas pivot_table command!

We'll use some info from http://www.gregreda.com/2013/10/26/working-with-pandas-dataframes/ to load the MovieLens 100k (http://grouplens.org/datasets/movielens/) dataset into a dataframe.

Read this: http://pandas.pydata.org/pandas-docs/stable/reshaping.html


In [3]:
import pandas as pd
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('c:/users/serafeim/desktop/imdb/ml-100k/u.user', sep='|', names=u_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('c:/users/serafeim/desktop/imdb/ml-100k/u.data', sep='\t', names=r_cols, encoding='latin-1')
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('c:/users/serafeim/desktop/imdb/ml-100k/u.item', sep='|', names=m_cols, usecols=range(5), encoding='latin-1')

In [4]:
movie_ratings = pd.merge(movies, ratings)
lens = pd.merge(movie_ratings, users)


In [5]:
lens.head()

,movie_id,title,release_date,video_release_date,imdb_url,user_id,rating,unix_timestamp,age,sex,occupation,zip_code
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,308,4,887736532,60,M,retired,95076
1,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,308,5,887737890,60,M,retired,95076
2,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),308,4,887739608,60,M,retired,95076
3,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,308,4,887738847,60,M,retired,95076
4,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),308,5,887736696,60,M,retired,95076


In [6]:
droplens = lens.drop(['video_release_date', 'imdb_url'], 1)
droplens['release_year']=droplens['release_date'].apply(lambda x: str(x).split('-')[2] if len(str(x).split('-'))>2 else 0)
droplens.head()

,movie_id,title,release_date,user_id,rating,unix_timestamp,age,sex,occupation,zip_code,release_year
0,1,Toy Story (1995),01-Jan-1995,308,4,887736532,60,M,retired,95076,1995
1,4,Get Shorty (1995),01-Jan-1995,308,5,887737890,60,M,retired,95076,1995
2,5,Copycat (1995),01-Jan-1995,308,4,887739608,60,M,retired,95076,1995
3,7,Twelve Monkeys (1995),01-Jan-1995,308,4,887738847,60,M,retired,95076,1995
4,8,Babe (1995),01-Jan-1995,308,5,887736696,60,M,retired,95076,1995


In [72]:
droplens[droplens['release_year'] > '1995'].pivot_table(
    columns=['rating', 'sex',],
    index=[ 'release_year'],
    values=[ 'age'], 
    fill_value=0,
    margins=True 
)


age                                              \
rating                1                     2                     3   
sex                   F          M          F          M          F   
release_year                                                          
1996          30.486239  30.373198  32.781437  31.576959  33.086124   
1997          30.333333  30.795053  31.926647  33.264409  32.638767   
1998          25.105263  31.028169  34.205882  36.059524  32.360000   
All           30.181609  30.593199  32.400000  32.404891  32.857241   

                                                                                
rating                           4                     5                   All  
sex                   M          F          M          F          M             
release_year                                                                    
1996          32.718188  32.757335  32.178464  30.494777  32.073340  32.126852  
1997          33.808467  33.224982  33.826004  32.225541  33.025606  33.086874  
1998          34.153061  32.845238  33.860656  33.785714  34.089431  33.403633  
All           33.208988  32.970278  32.986571  31.375121  32.611830  32.598420

In [178]:
droplens[droplens['release_year'] > '1995'].pivot_table(index=['rating'], values=['age'], columns=['sex'] ).dtypes

     sex
age  F      float64
     M      float64
dtype: object

How to use the pivot!

In [176]:

droplens[droplens['release_year'] > '1995'].head().pivot(index='movie_id', columns='age', values='rating')

age,60
movie_id,
15,3
17,4
21,3
22,4
23,5


NameError: name 'df' is not defined